In [1]:
import json
import os
import urllib

In [2]:
def download_and_load_file(file_path, url):
    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode('utf-8')
        
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(text_data)
    
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    return data

In [3]:
# instruction fine-tuning dataset based on the book

file_path = 'instruction-data.json'
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/01_main-chapter-code/instruction-data.json"
)

data = download_and_load_file(file_path, url)
print(f'Total Entries: {len(data)}')

Total Entries: 1100


In [4]:
print(f'Example:\n{data[50]}')

Example:
{'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}


In [ ]:
print(f'Example:\n{data[999]}')  # "input" field may be empty in the JSON

Example:
{'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."}
